# PLAN
## Initial
#### Safe bet to use Alpha Vantage to develop Model
## Final Step
#### Once ready to do extensive tranning with Multiple Stocks we will need to use SEC EDGAR most likely (need to figure that out) or pay for like a month subscription of Alpha Vantage

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta


# Using FMP
# - This is more accurate and consitent! Yahoo is web scrapping
# - Provides the financial statetement for a broader range then yahoo finance

# Key which is given from my FMP account. (This key is needed to make requests) Maximum of 200 calls/day
API_KEY = "wUvM2M29ZVxHDvK8IRp2P7iyrT8uhQG4"

ticker = "AAPL"

end_date = datetime(2024, 6, 1)
start_date = end_date - timedelta(days=50)

url_hist_stock_price = f"https://financialmodelingprep.com/stable/historical-price-eod/light?symbol={ticker}&from={start_date.date()}&to={end_date.date()}&apikey={API_KEY}"

# Historical Data Request
response_hist_stock_price = requests.get(url_hist_stock_price)

json_response_hist_stock_price = response_hist_stock_price.json()


In [2]:
closing_price = pd.DataFrame(json_response_hist_stock_price)

# Convert 'date' to datetime type
closing_price['date'] = pd.to_datetime(closing_price['date'])

# Keep only 'date' and 'price' columns
closing_price = closing_price[['date', 'price']]

# Set 'date' as the index
closing_price.set_index('date', inplace=True)

print(closing_price)

             price
date              
2024-05-31  192.25
2024-05-30  191.29
2024-05-29  190.29
2024-05-28  189.99
2024-05-24  189.98
2024-05-23  186.88
2024-05-22  190.90
2024-05-21  192.35
2024-05-20  191.04
2024-05-17  189.87
2024-05-16  189.84
2024-05-15  189.72
2024-05-14  187.43
2024-05-13  186.28
2024-05-10  183.05
2024-05-09  184.57
2024-05-08  182.74
2024-05-07  182.40
2024-05-06  181.71
2024-05-03  183.38
2024-05-02  173.03
2024-05-01  169.30
2024-04-30  170.33
2024-04-29  173.50
2024-04-26  169.30
2024-04-25  169.89
2024-04-24  169.02
2024-04-23  166.90
2024-04-22  165.84
2024-04-19  165.00
2024-04-18  167.04
2024-04-17  168.00
2024-04-16  169.38
2024-04-15  172.69
2024-04-12  176.55


In [3]:
# ALPHA VANTAGE EXPERIMENTING (https://www.alphavantage.co/documentation/#)
# gives some insight into the different attributes without sending demo request: https://documentation.alphavantage.co/FundamentalDataDocs/gaap_documentation.html

API_KEY = "FJLURMBXQEXWW9PP"

ticker = "AAPL"

def getInfo(function:str,ticker:str):
    url = f"https://www.alphavantage.co/query?function={function.upper()}&symbol={ticker}&apikey={API_KEY}"
    response = requests.get(url)
    return response.json()

In [21]:
# Requests
r_income_statements = getInfo("income_statement",ticker)
r_balance_sheet = getInfo("balance_sheet",ticker)
r_cash_flow = getInfo("cash_flow",ticker)

In [68]:
# Dataframes
df_income_statement = pd.DataFrame(r_income_statements["quarterlyReports"])
df_income_statement = df_income_statement.rename(columns={"fiscalDateEnding": "dates"})
df_income_statement.set_index("dates", inplace=True)
df_income_statement = df_income_statement.apply(pd.to_numeric, errors='coerce')
df_income_statement = df_income_statement.fillna(0)



df_balance_sheet = pd.DataFrame(r_balance_sheet["quarterlyReports"])
df_balance_sheet = df_balance_sheet.rename(columns={"fiscalDateEnding": "dates"})
df_balance_sheet.set_index("dates", inplace=True)
df_balance_sheet = df_balance_sheet.apply(pd.to_numeric, errors='coerce')
df_balance_sheet = df_balance_sheet.fillna(0)


df_cash_flow = pd.DataFrame(r_cash_flow["quarterlyReports"])
df_cash_flow = df_cash_flow.rename(columns={"fiscalDateEnding": "dates"})
df_cash_flow.set_index("dates", inplace=True)
df_cash_flow = df_cash_flow.apply(pd.to_numeric, errors='coerce')
df_cash_flow = df_cash_flow.fillna(0)


In [26]:
# Available data
print("Income Statement\n",df_income_statement.columns)
print("Balance Sheet\n",df_balance_sheet.columns)
print("Cash Flow\n",df_cash_flow.columns)

Income Statement
 Index(['reportedCurrency', 'grossProfit', 'totalRevenue', 'costOfRevenue',
       'costofGoodsAndServicesSold', 'operatingIncome',
       'sellingGeneralAndAdministrative', 'researchAndDevelopment',
       'operatingExpenses', 'investmentIncomeNet', 'netInterestIncome',
       'interestIncome', 'interestExpense', 'nonInterestIncome',
       'otherNonOperatingIncome', 'depreciation',
       'depreciationAndAmortization', 'incomeBeforeTax', 'incomeTaxExpense',
       'interestAndDebtExpense', 'netIncomeFromContinuingOperations',
       'comprehensiveIncomeNetOfTax', 'ebit', 'ebitda', 'netIncome'],
      dtype='object')
Balance Sheet
 Index(['reportedCurrency', 'totalAssets', 'totalCurrentAssets',
       'cashAndCashEquivalentsAtCarryingValue', 'cashAndShortTermInvestments',
       'inventory', 'currentNetReceivables', 'totalNonCurrentAssets',
       'propertyPlantEquipment', 'accumulatedDepreciationAmortizationPPE',
       'intangibleAssets', 'intangibleAssetsExcludingG

In [79]:
# TODO: Adjust all metrics such that they can be mapped to that of closing price (i.e the dates)

# Often used terms
net_income = df_income_statement['netIncome']
equity = df_balance_sheet["totalShareholderEquity"]
outstandingShares = df_balance_sheet["commonStockSharesOutstanding"]
book_value = equity
book_value_per_share = book_value/outstandingShares
total_revenue =df_income_statement['totalRevenue']

# Profitability Metrics
EPS = (net_income-df_cash_flow["dividendPayoutPreferredStock"])/outstandingShares
ROE = net_income/equity
net_Profit_Margin = net_income/total_revenue

# Valuation Metrics--> MOST PROBLEMATIC HERE
P_E_Ratio = closing_price/EPS
P_B_Ratio = closing_price/book_value_per_share
P_S_Ratio = closing_price*outstandingShares/total_revenue

# Risk and Efficiency metrics
D_E_Ratio = df_balance_sheet["shortLongTermDebtTotal"]/equity
Current_Ratio = df_balance_sheet["totalCurrentAssets"]/df_balance_sheet["totalCurrentLiabilities"]

In [81]:

#print(df_balance_sheet.columns)

outstandingShares_str = df_balance_sheet["commonStockSharesOutstanding"]
outstandingShares = outstandingShares_str.apply(pd.to_numeric, errors='coerce')
df_metrics = pd.DataFrame()

print(type(EPS))
df_metrics = pd.concat([
    EPS.rename('EPS'),
    ROE.rename('ROE'),
    D_E_Ratio.rename('D_E_Ratio'),
    Current_Ratio.rename('Current_Ratio')

], axis=1).dropna()

print(df_metrics)

"""net_Profit_Margin.rename("Net_Profit_Margin"),
    P_E_Ratio.rename('P_E_Ratio'),
    P_B_Ratio.rename('P_B_Ratio'),
    P_S_Ratio.rename('P_S_Ratio'),
    D_E_Ratio.rename('D_E_Ratio'),
    Current_Ratio.rename('Current_Ratio')"""


<class 'pandas.core.series.Series'>
                 EPS       ROE  D_E_Ratio  Current_Ratio
dates                                                   
2025-03-31  1.645841  0.370980   1.469938       0.820870
2024-12-31  2.397883  0.544204   1.449999       0.922938
2024-09-30  0.956380  0.258753   1.872327       0.867313
2024-06-30  1.397430  0.321521   1.518618       0.952980
2024-03-31  1.528383  0.318570   1.409683       1.037102
...              ...       ...        ...            ...
2006-03-31  0.016667  0.047224   0.000000       2.407989
2005-12-31  0.023082  0.067422   0.000000       2.310474
2005-09-30  0.017567  0.057594   0.000000       2.861366
2005-06-30  0.013235  0.046767   0.000000       2.895293
2005-03-31  0.012085  0.045412   0.000000       2.598449

[81 rows x 4 columns]


'net_Profit_Margin.rename("Net_Profit_Margin"),\n    P_E_Ratio.rename(\'P_E_Ratio\'),\n    P_B_Ratio.rename(\'P_B_Ratio\'),\n    P_S_Ratio.rename(\'P_S_Ratio\'),\n    D_E_Ratio.rename(\'D_E_Ratio\'),\n    Current_Ratio.rename(\'Current_Ratio\')'